# NVDI PDP local Estimation

In [1]:
%pwd

'/mnt/d/15_Article/03_RStudio/PyCode_v241111'

In [2]:
%cd ..

/mnt/d/15_Article/03_RStudio


/home/linux/anaconda3/envs/tf/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package 

In [38]:
from glob import glob
from joblib import dump, load
import joblib
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold

## Load Data

In [29]:
ori_df_list = []
pdp_df_list = []

for i in list(range(1, 11)):
    X_pdp = pd.read_parquet(f'03_Results/PdpLocal/PDP_Local_{i}.parquet')
    length = int(X_pdp.shape[0]/11)
    ori_df_list.append(X_pdp.iloc[:length, :])
    
    y_ori_pred = X_pdp.iloc[:length, -1].to_list() * 11
    
    X_pdp['incre'] = [0] * length + [0.1] * length + [0.2] * length + [0.3] * length + [0.4] * length + [0.5] * length + [0.6] * length + \
    [0.7] * length + [0.8] * length + [0.9] * length + [1.0] * length

    X_pdp['Pred_y_change'] = np.array(X_pdp['Pred_y']) - np.array(y_ori_pred)

    pdp_df_list.append(X_pdp)

In [30]:
pdp_df = pd.concat(pdp_df_list)
ori_df = pd.concat(ori_df_list)

In [32]:
pdp_df.shape

(4214903, 36)

In [31]:
ori_df.shape

(383173, 34)

## Build Local Connection

In [33]:
ori_df

,year,lat,lon,female,age,high_stress,low_stress,easy_to_relax,good_for_living,live_environment_satefy,...,master,phd,income_indiv,NDVI,NTL,left,right,down,up,Pred_y
26633,2015,35.918037,140.013626,0,61,2,4,4,5,3,...,0,0,4.5,41.080367,7.513318,140.011118,140.015556,35.916033,35.920255,4.060290
107444,2015,35.604172,139.486008,0,62,3,3,4,4,3,...,0,0,5.5,43.755053,17.125202,139.484421,139.487282,35.603373,35.605078,3.955881
139014,2015,35.170216,138.682770,0,37,3,3,3,3,3,...,0,0,5.5,39.437845,10.424026,138.668373,138.698227,35.168500,35.171721,3.274137
136605,2015,34.741104,137.707062,0,52,4,5,3,5,3,...,0,0,4.5,36.496210,15.807176,137.693361,137.719389,34.740381,34.741859,3.851559
471562,2017,33.515968,130.407059,0,50,5,4,1,4,3,...,0,0,3.5,50.141388,10.332232,130.405769,130.408966,33.510625,33.520088,2.500626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295192,2016,35.694298,139.442795,0,54,2,2,4,4,3,...,0,0,2.5,31.113321,24.530603,139.441521,139.444061,35.693935,35.694692,3.667642
11296,2015,40.832371,140.712585,1,54,4,5,4,3,3,...,0,0,3.5,24.593328,6.966584,140.706610,140.718414,40.816635,40.894635,3.837474
150474,2015,35.252342,137.006027,1,51,4,5,2,4,3,...,0,0,4.5,37.217069,14.977743,137.002693,137.009438,35.250058,35.254816,3.489156
405310,2017,35.734009,139.552185,1,45,4,4,4,4,3,...,0,0,4.5,30.448610,28.393964,139.550919,139.553459,35.733593,35.734425,3.852766


In [35]:
ori_df.iloc[0, :]

year                       2015.000000
lat                          35.918037
lon                         140.013626
female                        0.000000
age                          61.000000
high_stress                   2.000000
low_stress                    4.000000
easy_to_relax                 4.000000
good_for_living               5.000000
live_environment_satefy       3.000000
community_attachment          4.000000
income                        4.000000
self_reported_health          4.000000
student                       0.000000
worker                        1.000000
company_owner                 0.000000
government_officer            0.000000
self_employed                 0.000000
professional                  0.000000
housewife                     0.000000
retired                       0.000000
unemployed                    0.000000
college_no_diploma            1.000000
bachelor                      0.000000
master                        0.000000
phd                      

In [37]:
pdp_df.columns

Index(['year', 'lat', 'lon', 'female', 'age', 'high_stress', 'low_stress',
       'easy_to_relax', 'good_for_living', 'live_environment_satefy',
       'community_attachment', 'income', 'self_reported_health', 'student',
       'worker', 'company_owner', 'government_officer', 'self_employed',
       'professional', 'housewife', 'retired', 'unemployed',
       'college_no_diploma', 'bachelor', 'master', 'phd', 'income_indiv',
       'NDVI', 'NTL', 'left', 'right', 'down', 'up', 'Pred_y', 'incre',
       'Pred_y_change'],
      dtype='object')

In [54]:
for line_row in range(ori_df.shape[0]):
    left = ori_df.iloc[line_row, 29]
    right = ori_df.iloc[line_row, 30]
    down = ori_df.iloc[line_row, 31]
    up = ori_df.iloc[line_row, 32]

    this_point_df = pdp_df[(pdp_df['lat']>down) & (pdp_df['lat']<up) & (pdp_df['lon']>left) & (pdp_df['lon']>right)]
    this_point_df = this_point_df[['incre', 'Pred_y_change']]

    pdp_local = this_point_df['Pred_y_change'].groupby(this_point_df['incre']).mean()
    pdp_local = pd.DataFrame(pdp_local).reset_index()
    
    X = pdp_local[['incre']]
    y = pdp_local['Pred_y_change']

    model = LinearRegression()
    model.fit(X, y)

    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    print("R^2 Score:", r2)

    #print(model.coef_, model.intercept_)
    if line_row == 10:
        break

R^2 Score: 0.20093833406501116
R^2 Score: 0.1354165274850354
R^2 Score: 0.8637872364528931
R^2 Score: 0.5519715662687084
R^2 Score: 0.9775298881980751
R^2 Score: 0.9871896030530652
R^2 Score: 0.6347010987764988
R^2 Score: 0.9329762824496927
R^2 Score: 0.9676424598479922
R^2 Score: 0.28881760355731567
R^2 Score: 0.8790946298463622
